# Rating Product

In [ ]:
import os
from IPython.display import Image, display

file_path = '/kaggle/input/rating-image/Rating ProductSortingReviewsin.png'

display(Image(filename=file_path))

## Importing Libraries and Display Settings

In [2]:
import pandas as pd
import math
import scipy.stats as st

#Display Settings
pd.set_option('display.max_columns',500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Examining the Dataset

In [3]:
df_ = pd.read_csv('/kaggle/input/amazon-review/amazon_review.csv')
df= df_.copy()
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [7]:
df.isnull().sum()

,0
reviewerID,0
asin,0
reviewerName,1
helpful,0
reviewText,1
overall,0
summary,0
unixReviewTime,0
reviewTime,0
day_diff,0


In [8]:
df.dropna(inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4913 entries, 1 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4913 non-null   object 
 1   asin            4913 non-null   object 
 2   reviewerName    4913 non-null   object 
 3   helpful         4913 non-null   object 
 4   reviewText      4913 non-null   object 
 5   overall         4913 non-null   float64
 6   summary         4913 non-null   object 
 7   unixReviewTime  4913 non-null   int64  
 8   reviewTime      4913 non-null   object 
 9   day_diff        4913 non-null   int64  
 10  helpful_yes     4913 non-null   int64  
 11  total_vote      4913 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 499.0+ KB


In [15]:
df['overall'].value_counts().sort_index(ascending=False)

,count
overall,
5.00000,3921
4.00000,526
3.00000,142
2.00000,80
1.00000,244


In [16]:
df['overall'].mean()

4.58762466924486

In [20]:
print('Last 90 Days Score:', df.loc[df['day_diff'] <= 90, 'overall'].mean())
print('90-180 Days Score:', df.loc[(df['day_diff'] > 90) & (df['day_diff'] < 180), 'overall'].mean())
print('180-360 Days Score:', df.loc[(df['day_diff'] >= 180) & (df['day_diff'] < 360), 'overall'].mean())
print('360+ Days Score:', df.loc[df['day_diff'] >= 360, 'overall'].mean())

Last 90 Days Score: 4.782383419689119
90-180 Days Score: 4.651162790697675
180-360 Days Score: 4.683357879234168
360+ Days Score: 4.523025210084033


* The product's score has been getting better.

* To observe this better, we need to do a time-based scoring.

## Time-Based Scoring

In [24]:
def time_based_scoring(dataframe, w1=30, w2=27, w3=23, w4=20):
    return ((df.loc[df['day_diff'] <= 90, 'overall'].mean())*w1 + \
     (df.loc[(df['day_diff'] > 90) & (df['day_diff'] < 180), 'overall'].mean())*w2 + \
     (df.loc[(df['day_diff'] >= 180) & (df['day_diff'] < 360), 'overall'].mean())*w3+ \
     (df.loc[df['day_diff'] >= 360, 'overall'].mean())*w4) / 100

time_based_scoring(df)

4.672306333635773

* Weights w1, w2, w3, w4 can be changed according to the product and situation.
* Also, values ​​such as 90, 180 and 360 days can be changed according to the trend status of the product.

# Sorting Reviews

In [25]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0
5,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,"[0, 0]",It's mini storage. It doesn't do anything els...,5.00000,Not a lot to really be said,1367193600,2013-04-29,588,0,0


total_vote = helpful_yes + helpful_no

In [26]:
df['helpful_no'] = df['total_vote'] - df['helpful_yes']

## Finding the Difference Between Positive and Negative votes, Their Average and Wilson Lower Bound

**Wilson Lower Bound:** It is a correction factor that adjusts the lower limit of the confidence interval to account for any bias in the sample.

In [27]:
def score_pos_neg_diff(up, down):
    return up - down

def average_pos_neg(up,down):
    if (up + down) != 0:
        return up / (up + down)
    else:
        return 0

def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)


In [33]:
df['score_pos_neg_diff'] = df.apply(lambda x: score_pos_neg_diff(x['helpful_yes'], x['helpful_no']), axis=1)
df['score_average_rating'] = df.apply(lambda x: average_pos_neg(x['helpful_yes'], x['helpful_no']), axis=1)
df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'], x['helpful_no']), axis=1)

In [36]:
df.sort_values(by='wilson_lower_bound', ascending=False).head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.91837,0.80811


* When we examine the first 4 comments, we see that the total votes are big. The wisdom of the crowd has been taken into account.

* However, when we look at the 5th comment, we see that it only has 49 votes. But it was found to be very useful and rose to the top.
